In [ ]:
#### import global modules
import os
import sys
import pandas as pd
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Network_Query='''


with Customer_details as
 (
SELECT a.customer_id,
  FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` as a
left join `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_terminated_details` as b
on a.customer_id=b.customer_id WHERE b.customer_id IS NULL
and a.last_updt_ts=(SELECT MAX(st1.last_updt_ts) FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` AS st1) 
),


-- OLDER METHOD FOR MAX RECORD
--  signal_info_interim as (
--    select customer_id, network_desc, history_date_utc
--          ,row_number() over (partition by customer_id order by history_date_utc desc) as rn
--    from `cio-datahub-enterprise-pr-183a.src_adc.bq_signal_rating_history`
-- ),

-- OLDER METHOD FOR MAX RECORD

-- signal_info as
-- (SELECT * FROM signal_info_interim where rn=1),




-- NEW METHOD FOR MAX RECORD
signal_info_final as
(select customer_id,network_id, network_desc,signaling_rating_id,signaling_rating_name,sinaling_rating_logic, history_date_utc as signal_rating_date from `cio-datahub-enterprise-pr-183a.src_adc.bq_signal_rating_history` 
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY history_date_utc DESC) = 1)
-- ,


select cd.* ,sig.network_id,sig.network_desc,sig.signaling_rating_id,sig.signaling_rating_name,sig.sinaling_rating_logic, sig.signal_rating_date from Customer_details cd 
left join signal_info_final sig 
on  cd.customer_id=sig.customer_id
order by cd.customer_id




'''

In [ ]:
Customer_Network_details_DF= extract_bq_data(bq_client,sql=Network_Query)

In [ ]:
Customer_Network_details_DF.info()

In [ ]:
Customer_Network_details_DF.isnull().sum() * 100 / len(Customer_Network_details_DF)

In [ ]:
Customer_Network_details_DF.head()

In [ ]:
Customer_Network_details_DF['network_desc'].fillna('Missing_network_information',inplace=True)

In [ ]:
Customer_Network_details_DF.value_counts('network_desc',normalize=True)*100

In [ ]:
job_config_Network = bigquery.job.LoadJobConfig()
job_config_Network.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Network_BQ_table = 'ADC.Customer_Network_details'

job_device_type= bq_client.load_table_from_dataframe(Customer_Network_details_DF, Network_BQ_table,job_config=job_config_Network)